In [4]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification

In [6]:
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                             random_state=0, shuffle=False)


In [7]:
# %timeit sirve para evaluar el tiempo de ejecucion
event = pd.read_csv('events_up_to_01062018.csv')
label_train = pd.read_csv('labels_training_set.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

#projects_2017.head()trocafone_kaggle_test
#projects_2017

In [8]:
event.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,"6,888.00",iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,"11,890.00",iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,"7,517.00",LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#feature engine trucho
eventos=event[['person','sku']].fillna(0);
eventos.head()

,person,sku
0,4886f805,"9,288.00"
1,ad93850f,304.00
2,0297fc1e,"6,888.00"
3,2d681dd8,"11,890.00"
4,cccea85e,"7,517.00"


In [10]:
label_train.head(5)

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [41]:
label_train.groupby('person').agg({'person':'count'}).sort_values('person',ascending=True)

,person
person,
0008ed71,1
ac091252,1
ac04df6c,1
abffbb2f,1
abf91c38,1
abf81db4,1
abf1e6f0,1
ac102edc,1
abf0cf53,1


In [49]:
labe_train_con_feature=pd.merge(label_train,eventos,how='inner',on='person')
labe_train_con_feature=labe_train_con_feature.drop_duplicates(subset='person',keep='first')
labe_train_con_feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 1171874
Data columns (total 3 columns):
person    19414 non-null object
label     19414 non-null int64
sku       19414 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 606.7+ KB


In [12]:
test.head(5)


,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [51]:
#feature engieri del test,misma columna train,pero sin label.
 
test_feature_engenieri=pd.merge(test,eventos,how='inner',on='person')
test_feature_engenieri=test_feature_engenieri.drop_duplicates(subset='person',keep='first')
labe_train_con_feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 1171874
Data columns (total 3 columns):
person    19414 non-null object
label     19414 non-null int64
sku       19414 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 606.7+ KB


In [52]:
string_columns=labe_train_con_feature.columns
string_columns

Index(['person', 'label', 'sku'], dtype='object')

In [53]:
#hash
#Transformo texto a numeros
for f in string_columns:
         labe_train_con_feature[f] = labe_train_con_feature.apply(lambda row:  hash(row[f]) if not(pd.isnull(row[f]))
         else np.nan, axis = 1)

In [54]:
labe_train_con_feature.head(5)

,person,label,sku
0,9185933318720544067,0,0
68,9034762767663080449,0,0
70,259413289375064484,0,3359
166,-4951965066334574255,0,0
203,2655864631403522232,0,0


In [55]:
# es entrenar el modelo en  ramdom forest

In [56]:
#formato del train

In [57]:
train_label = labe_train_con_feature['label']
train_label.head()
 

0      0
68     0
70     0
166    0
203    0
Name: label, dtype: int64

In [58]:
train = labe_train_con_feature.drop('label', 1)
train.head()
train=train[['person','sku']] 
train.head()

,person,sku
0,9185933318720544067,0
68,9034762767663080449,0
70,259413289375064484,3359
166,-4951965066334574255,0
203,2655864631403522232,0


In [59]:
forest = RandomForestRegressor(n_estimators = 20, n_jobs = -1,random_state =3 ,
                                max_features = "auto", min_samples_leaf = 6)


In [60]:
forest.fit(train, train_label)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=6, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=3, verbose=0, warm_start=False)

In [61]:
string_columns=test_feature_engenieri.columns
string_columns

Index(['person', 'sku'], dtype='object')

In [62]:
#hash del test
#hash
#Transformo texto a numeros
for f in string_columns:
         test_feature_engenieri[f] = test_feature_engenieri.apply(lambda row:  hash(row[f]) if not(pd.isnull(row[f]))
         else np.nan, axis = 1)

In [63]:
test_feature_engenieri.head()

,person,sku
0,1612889871667063446,9288
9,-3473783696745989064,6888
576,-7874171715875044824,11890
602,8220131127206862214,7517
1438,2768383520292848369,0


In [64]:
#predecimos
prediction = forest.predict(test_feature_engenieri)


In [65]:
 len(prediction)
    

19415

In [74]:
ids = test['person'].values
dataKaggle = pd.DataFrame(ids ,columns={'person'})
dataKaggle.head(5)

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [67]:
len(dataKaggle)

19415

In [75]:
#Aca agregamos una columna que seria se postulop
dataKaggle['label'] = prediction

In [81]:
 dataKaggle.nlargest(5,'label'  )

,person,label
138,22f1d021,0.58
706,38b66a87,0.56
2076,ad1f751c,0.53
3489,cd4168bc,0.53
8903,36c224c9,0.53


In [79]:
dataKaggle.to_csv("predictionRandonForest.csv", index = False)